In [2]:
"""
Data Preprocessing
"""

import os
import re
import pandas as pd

# read train data
file_dir = '/Users/dnhb/PycharmProjects/SML_Ass1'
file_name = "train.txt"

def parse(data:list)->list: 
    # Python Notation: https://www.python.org/dev/peps/pep-3107/
    """
    Prase each line of the data
    Return a nested list, such as [[id1, id2, id2,...],...]
    """
    parsed_data = []
    for line in data:
        # remove \n at the end of each line
        line = re.sub(r"(?<=\d)\n", "", line)
        # split IDs by \t
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        parsed_data.append(line)
    return parsed_data

with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = parse(train_set)
    
# read test data
test_name = "test-public.txt"
with open(os.path.join(file_dir, test_name)) as f:
    test_set = f.readlines()
    test_set = parse(test_set)
    test_set.pop(0)

# turn ID into int
for i in range(0, len(test_set)):
    test_set[i][0] = int(test_set[i][0])
    
print("Data has been read.")

Data has been read.


In [3]:
"""
Sigma Data For Similarity
"""

from collections import defaultdict

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

# construct SigmaOut dic for each node
SigmaOut = defaultdict(set)

def getSigmaOut(node:str, graph:dict) -> set:
    SigmaOut = set()
    try:
        for item in graph[node]:
            SigmaOut.add(item)
        return SigmaOut
    except:
        return SigmaOut

for kw in train_dic:
    SigmaOut[kw] = getSigmaOut(kw, train_dic)
    
print("SigmaOut done.\n")

# construct SigmaIn dic for each node
SigmaIn = defaultdict(set)

for kw in train_dic:
    for followee in train_dic[kw]:
        if SigmaIn[followee]:
            SigmaIn[followee].add(kw)
        else:
            SigmaIn[followee] = {kw}

def getSigma(node:str) -> set:
    return SigmaIn[node].union(SigmaOut[node])

print("SigmaIn done.")

SigmaOut done.

SigmaIn done.


In [4]:
"""
Similarity Functions
"""

import math

    
def adar1(a:str, b:str) -> float:
    score = 0
    SigmaA = getSigma(a)
    SigmaB = getSigma(b) 
    SigmaV = SigmaA.intersection(SigmaB)
    if SigmaV:
        for v in SigmaV:
            if getSigma(v) and len(getSigma(v)) != 1:
                   score += (1/math.log(len(getSigma(v))))
            else:
                   pass
        return score
    else:
        return 0

def adar2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += adar1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0
    
def adar3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += adar1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
  
def jaccard1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / len(SigmaA.union(SigmaB)))

def jaccard2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += jaccard1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def jaccard3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += jaccard1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
    
def cosine1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / (len(SigmaA)*len(SigmaB)))


def cosine2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += cosine1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def cosine3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += cosine1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0

def common_neighbors1(a: str, b: str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return len(SigmaA.intersection(SigmaB))

def common_neighbors2(a: str, b: str) -> float:
     score = 0
     SigmaOutA = SigmaOut[a]
     if SigmaOutA:
         for node in SigmaOutA:
             score += common_neighbors1(b, node)
         return float(score / len(SigmaOutA))
     else:
         return 0

def common_neighbors3(a: str, b: str) -> float:
     score = 0
     SigmaInB = SigmaIn[b]
     if SigmaInB:
         for node in SigmaInB:
             score += common_neighbors1(a, node)
         return float(score / len(SigmaInB))
     else:
         return 0

def knn1(a:str, b:str) -> float:
    """
    knn1 = $w_{ab} = w_a^out$
    """
    return 1/math.sqrt(1+len(SigmaOut[a]))

def knn2(a:str, b:str) -> float:
    """
    knn2 = $w_{ab} = w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaIn[b]))

def knn3(a:str, b:str) -> float:
    """
    knn3 = $w_{ab} = w_a^out  w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaOut[a])) + 1/math.sqrt(1+len(SigmaIn[b]))

def knn4(a:str, b:str) -> float:
    """
    knn4 = $w_{ab} = w_a^out * w_b^in$
    """
    wAOut = 1/math.sqrt(1+len(SigmaOut[a]))
    wBIn = 1/math.sqrt(1+len(SigmaIn[b]))
    return  wAOut* wBIn

In [6]:
"""
This cell only works when reading data from local 
"""

# read X_train data
score_name = "X_train3Score.csv"
with open(os.path.join(file_dir, score_name)) as f:
    X_train = f.readlines()
    X_train = parse(X_train)
    X_train.pop(0)
    
for i in range(0, len(X_train)):
    X_train[i] = X_train[i][0].split(",")
    for j in range(0, 7):
        X_train[i][j] = float(X_train[i][j])
        

# construct Y
positive_label = [1] * 10000
negative_label = [0] * 10000
Y_train = []
Y_train.extend(positive_label)
Y_train.extend(negative_label)

"""
Step:
1. run this cell
2. run the *assert block*

if need to select feature, then
1. run the X_trian1 cell, select the features
2. edit and run *getFeature* cell
3. run the paparmeters of RF
4. gridsearch and so on 
"""

'\nStep:\n1. run this cell\n2. run the *assert block*\n\nif need to select feature, then\n1. run the X_trian1 cell, select the features\n2. edit and run *getFeature* cell\n3. run the paparmeters of RF\n4. gridsearch and so on \n'

In [7]:
assert(len(X_train) == len(Y_train))

In [7]:
"""
Select Features
"""

# "adar1",  "jaccard2", "jaccard3", "cosine2", "cosine3"
X_train1 = []
for item in X_train:
    new = []
    new.extend(item[:7])
    #new.extend(item[2:4])
    #new.extend(item[5:])
    X_train1.append(new)
    
X_train = X_train1

In [5]:
"""
Get Feature
"""

def getFeature(pair:tuple) -> float:
    node1 = pair[0]
    node2 = pair[1]
    result = []
    result.append(adar1(node1,node2))
    #result.append(salton(node1,node2))
    #result.append(allocation(node1,node2))
    result.append(jaccard1(node1,node2))
    result.append(jaccard2(node1,node2))
    result.append(jaccard3(node1,node2))
    result.append(cosine1(node1,node2))
    result.append(cosine2(node1,node2))
    result.append(cosine3(node1,node2))
    #result.append(knn1(node1,node2))
    #result.append(knn2(node1,node2))
    #result.append(knn3(node1,node2))
    #result.append(knn4(node1,node2))
    #result.append(common_neighbors1(node1,node2))
    #result.append(common_neighbors2(node1,node2))
    #result.append(common_neighbors3(node1,node2))
    #result.append(AA(node1,node2))
    #result.append(CNPA(node1,node2))
    print(result)
    return result

In [34]:
"""
Train RandomForestClassifier
Using GridSearchCV to find the best estimators
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the random grid for rfc
para_rfc = {'n_estimators': [50, 100],
                 "criterion": ["gini", "entropy"],
                 'max_depth': [50,70, None],
                 'min_samples_split': [2, 3, 5, 8, 10],
                 'min_samples_leaf': [1, 2, 4, 6, 8]}

rfc = RandomForestClassifier(bootstrap=True, criterion='gini',
           max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False )
grid_rfc = GridSearchCV(estimator = rfc, param_grid = para_rfc, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_rfc.fit(X_train, Y_train)
model = grid_rfc.best_estimator_
model.fit(X_train, Y_train)
print (model)
print ("rfc train success!")

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  5.1min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
rfc train success!


In [8]:
"""
Best estimators of RFC with features adar1, jaccard 1-3, cos 1-3
"""

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
model.fit(X_train, Y_train)
print ("rfc train success!")

rfc train success!


In [ ]:
"""
Train Support Vector Classification
Using GridSearchCV to find the best estimators
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

para_svc = {"C": [1, 3, 5],
               "kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']}

svc = RandomForestClassifier(bootstrap=True, criterion='gini',
           max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False )
grid_svc = GridSearchCV(estimator = svc, param_grid = para_svc, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_svc.fit(X_train, Y_train)
model = grid_svc.best_estimator_
model.fit(X_train, Y_train)
print (model)
print ("svc train success!")

In [ ]:
"""
Train LogisticRegression
Using GridSearchCV to find the best estimators
"""

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the random grid for rfc
para_lgr = {'penalty': ["l1", "l2", "elasticnet", "none"],
                 'tol': [1e-4, 1e-6, 1e-8],
                 'C': [1, 3, 5],
                 'fit_intercept': [True, False],
                 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                 'max_iter': [100, 200],
                 'multi_class': ['auto'],
                 'n_jobs': [1]}    

lgr = LogisticRegression(random_state=0)
grid_lgr = GridSearchCV(estimator = lgr, param_grid = para_lgr, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_lgr.fit(X_train, Y_train)
model = grid_lgr.best_estimator_
model.fit(X_train, Y_train)
print (model)
print ("lgr train success!")

Fitting 3 folds for each of 720 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.2min


In [ ]:
"""
Best estimators of LGR with features adar1, jaccard 1-3, cos 1-3
"""

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=1, penalty='none', random_state=0,
                   solver='newton-cg', tol=0.0001, verbose=0, warm_start=False)
model.fit(X_train, Y_train)

In [9]:
import multiprocessing as mp
import time

if __name__ == '__main__':
    
    def getPro(record:list) -> list:
        feature = getFeature((record[1], record[-1]))
        prediction = model.predict_proba([feature])[0][1]
        label = model.predict([feature])[0]
        pro = [record[0], prediction, label]
        print(pro)
        return pro
    
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    result = pool.map(getPro, test_set)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    title = ["Id", "Predicted", "Label"]
    test_pd = pd.DataFrame(columns = title, data = result)
    test_pd.to_csv('/Users/dnhb/PycharmProjects/SML_Ass1/RFC2.csv',encoding='utf-8')

[0.21933342447606208, 0.005154639175257732, 0.0010560980236248875, 0.00576653084902314, 0.001038961038961039, 0.0002158649279589648, 1.5493731237062647e-05]
[0, 0.0, 0.0006682103297357281, 1.8060941139843826e-05, 0.0, 0.00034322807471536454, 7.228122423328354e-08]
[316, 0.0, 0]
[442, 0.0, 0]
[0, 0.0, 0.004756335876775657, 0.0005961701829318617, 0.0, 0.0011086098198699584, 3.067381859917711e-05]
[1, 0.0, 0]
[0, 0.0, 0.0004547391940738875, 0.007885478256716273, 0.0, 5.10988236771775e-05, 0.0002286423081153753]
[2, 0.0, 0]
[0.4627293341056383, 0.011152416356877323, 0.006910701545848951, 0.024294678294895095, 0.0006920415224913495, 0.0005541850751437438, 0.00018663650315648496]
[0.822336943717954, 0.02147239263803681, 0.01244905313913601, 0.006981832564143511, 0.0004018369690011481, 0.0005297209380207124, 3.7517070022812676e-05]
[3, 0.06126190476190477, 0]
[127, 0.06575, 0]
[0, 0.0, 0.0, 5.02373715807189e-05, 0.0, 0.0, 5.58585883976126e-07]
[128, 0.0, 0]
[1.0980735409705344, 0.013698630136

[65, 0.15360317460317457, 0]
[29.89794923735107, 0.18254764292878636, 0.04679525197987243, 0.06124789248925164, 0.000530893943725242, 0.0008230413547663114, 0.0002354258927378303]
[258, 1.0, 1]
[0, 0.0, 0.0027146610732419704, 7.572855279934165e-05, 0.0, 0.0006621860469981223, 6.67376656996589e-07]
[326, 0.0, 0]
[7.2420165395727185, 0.1040339702760085, 0.03509928038377915, 0.03126384551199464, 0.0007985788555876074, 0.0008058146526775436, 0.00016642458555895203]
[0, 0.0, 0.0007490310165941996, 0.0006286491974390266, 0.0, 9.222095759689695e-05, 5.8356978613035805e-06]
[134, 0.9468333333333334, 1]
[259, 0.0, 0]
[9.55319600231659, 0.08213820078226858, 0.03708215296325779, 0.015602962057144473, 0.00040492595639654463, 0.0005804369562858416, 7.834012390034124e-05]
[0.9557282025623033, 0.01951219512195122, 0.05634277188256289, 0.04554460018142172, 0.00392156862745098, 0.020121664006326687, 0.0010484155040296956]
[384, 0.9593333333333334, 1]
[135, 1.0, 1]
[0.12821592138496524, 0.00213219616204

[1.0298367836950175, 0.02412280701754386, 0.013075067757492061, 0.002167287427819258, 0.00029546065001343, 0.00030935171328587413, 7.108676251315247e-06]
[193, 0.0, 0]
[0, 0.0, 0.005083832334797554, 5.678159649695963e-05, 0.0, 0.00020046380790227558, 9.17588014454053e-07]
[0, 0.0, 0.011337564732222773, 0.001167309945543843, 0.0, 0.0006269754528367193, 5.7439213904946165e-06]
[461, 0.0, 0]
[334, 0.0, 0]
[0, 0.0, 0.0003287983662890618, 0.00010565494715320857, 0.0, 0.00016671936964153948, 3.4633627961208005e-07]
[10.274761308510321, 0.04463208685162847, 0.021550684566970558, 0.029747165968397687, 0.00013864662161276, 0.00014420864241662223, 0.00011364153427992775]
[268, 0.0, 0]
[388, 0.98875, 1]
[0.2176536083726855, 0.008130081300813009, 0.0071538206316898185, 0.006909091080790497, 0.00013605442176870748, 0.00020837453581537606, 7.044183174571858e-05]
[335, 0.005, 0]
[0, 0.0, 0.01086563477044678, 0.019632939915582844, 0.0, 0.0008797852060049671, 5.058095135683228e-06]
[143, 0.051666666666

[271, 0.2971666666666667, 0]
[0.0897643439842049, 0.001644736842105263, 0.010781835144750266, 0.003661147501991882, 0.00010539629005059021, 0.0007971638935756519, 6.427931436284522e-06]
[197, 0.0, 0]
[0, 0.0, 9.572530311883383e-05, 0.00045641259698767686, 0.0, 4.800833106012607e-05, 4.8702879963635184e-06]
[198, 0.0, 0]
[0, 0.0, 0.004865553843954178, 2.678324921634084e-05, 0.0, 0.00024012335513554615, 4.687588682643133e-07]
[29, 0.0, 0]
[0, 0.0, 0.031653542935579614, 4.129548583915907e-05, 0.0, 0.0013514253826259859, 2.9586067797939304e-06]
[154, 0.00875, 0]
[0.33729498626639165, 0.007894736842105263, 0.0053441750174695445, 0.0008190095453711455, 8.241758241758242e-05, 0.00010074191030474879, 4.299363830655095e-06]
[341, 0.0, 0]
[0, 0.0, 0.00046624820870387525, 0.011395674325381222, 0.0, 0.00010153730817576416, 2.109094351112229e-05]
[30, 0.0, 0]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[31, 0.0, 0]
[0.07796806287802142, 0.008620689655172414, 0.002833135675752537, 0.0009194535730035492, 0.000

[402, 1.0, 1]
[0, 0.0, 0.0021889125502341966, 0.0032180781552709303, 0.0, 5.379231425038181e-05, 9.574295995998472e-06]
[403, 0.0, 0]
[0, 0.0, 0.011115648315157248, 3.2777256255211587e-06, 0.0, 0.003852126730166922, 3.997577531976862e-08]
[404, 0.0, 0]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[405, 0.0, 0]
[0, 0.0, 0.014518740192005782, 0.0033146150798754196, 0.0, 0.0007357895486143874, 1.941290868892346e-05]
[0.1348914676310476, 0.034482758620689655, 0.019822484298917812, 0.0005840204864718559, 0.006944444444444444, 0.00477098675850637, 2.457624368168197e-05]
[406, 0.05583333333333333, 0]
[162, 0.01, 0]
[5.370404478711435, 0.42857142857142855, 0.16132286928987485, 0.25001725737565517, 0.017269736842105265, 0.007680224781829815, 0.010507857845999198]
[163, 1.0, 1]
[1.343990882375522, 0.0038529934795494963, 0.021969793066836617, 0.005970415066712535, 1.0951702989814915e-05, 0.0003595335781290292, 3.119010128919798e-06]
[278, 0.40983333333333327, 0]
[0, 0.0, 3.765925849489855e-05, 0.00484865770

...

[1638, 0.0, 0]
[0, 0.0, 0.0004874731405149975, 0.0002957294110047235, 0.0, 6.77239366394958e-05, 4.522923681040141e-06]
[1808, 0.0, 0]
[0.31827076777099816, 0.004601226993865031, 0.00649373935149192, 0.002164914415505475, 3.0629747610879685e-05, 0.00014615307171322777, 8.06571000967237e-06]
[0, 0.0, 9.073456017558622e-06, 0.003401639830256827, 0.0, 2.2748391049835965e-06, 2.148334985152498e-05]
[1954, 0.0, 0]
[0.19155782354835868, 0.01818181818181818, 0.10058728813750216, 0.040343798080135114, 0.0033333333333333335, 0.038322368741621135, 0.0008609663334446953]
[1849, 0.0, 0]
[1955, 0.9966666666666667, 1]
[0, 0.0, 9.306684045370083e-06, 0.0031835163761285133, 0.0, 3.1058358793646424e-06, 4.501970682938483e-05]
[1956, 0.0, 0]
[0.09040609003438033, 0.0019880715705765406, 0.008042114403471974, 0.0001917859920417673, 1.5753241229382945e-05, 0.00020415713918205412, 8.090054788948828e-07]
[1694, 0.0, 0]
[0.21625850913623576, 0.004319654427645789, 0.019833152353553155, 0.0046363005675279356, 0

[0.07751681639600151, 1.6821706730364864e-05, 0.003269148365350988, 0.02300783387383059, 1.6821706730364864e-05, 0.003269148365350988, 4.3448469577708454e-07]
[0.07383035914027619, 0.006896551724137931, 0.006496259622348851, 0.00016704766433272864, 0.00019592476489028212, 0.00026759763676159853, 3.078866033294076e-06]
[1958, 0.06866666666666667, 0]
[1490, 0.0, 0]
[1.6427129898066621, 0.028037383177570093, 0.009542298101051776, 0.08381880237963851, 0.0014516129032258066, 0.0005622068628341234, 0.0008491740605588149]
[1821, 0.5458333333333334, 1]
[0.86499766283403, 0.008915304606240713, 0.016193899456151663, 0.013259799556969163, 0.00014074595355383532, 0.0008044903817617634, 3.764207086831563e-05]
[1959, 0.47241666666666654, 0]
[0.14404745418365533, 0.006172839506172839, 0.022798757742688216, 0.005764517618948064, 0.00015220700152207003, 0.0011903124495085727, 7.469637454768007e-05]
[1960, 0.4339166666666667, 0]
[0.8189962342964474, 0.005263157894736842, 0.02009632212130457, 0.002472350

[0.3635685307972604, 0.000333555703802535, 0.005469499981433964, 0.0014036175430489275, 1.3393604553825548e-05, 4.3217483810300074e-05, 6.91089795669643e-05]
[1923, 0.0, 0]
[1861, 0.13666666666666666, 0]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1862, 0.0, 0]
[0.10256792305914779, 0.0031545741324921135, 0.003745321001281971, 0.00090544389550983, 0.0005341880341880342, 0.0011077397510737292, 2.9640479767536593e-06]
[1863, 0.0, 0]
[0, 0.0, 0.0020280087994347614, 0.01196478116329116, 0.0, 0.0002850407476657491, 0.00012780986829456426]
[1924, 0.0175, 0]
[5.42876599029407, 0.020810907786149982, 0.06565577261708778, 0.016170539354763053, 9.277031349968011e-05, 0.0007505753554535241, 8.996669963125013e-06]
[1499, 0.6571666666666667, 1]
[0, 0.0, 0.005376044124955958, 3.7285895740928045e-05, 0.0, 0.0001982976738630565, 8.483166496692302e-07]
[1500, 0.0, 0]
[0, 0.0, 0.00040275352138752403, 6.033974446870659e-06, 0.0, 7.164654124313033e-06, 3.7739156215154867e-07]
[1864, 0.0, 0]
[5.742533849301726, 0.13

[1.837805118788338, 0.001990049751243781, 0.027614030824155894, 0.03453699787628808, 2.1127752476642095e-05, 0.0005625707145151628, 8.255266276555945e-06]
[1937, 0.5508333333333334, 1]
[0.07383035914027619, 0.003134796238244514, 0.017479801177151977, 0.002137260326114961, 0.00010283833813245579, 0.00117230962490608, 1.3903655162717757e-05]
[1882, 0.003333333333333333, 0]
[0, 0.0, 0.0002422882629582655, 7.08504703556732e-05, 0.0, 0.00012329564258033792, 1.0686525599480602e-07]
[1976, 0.0, 0]
[0, 0.0, 0.006025355062627427, 0.00036939582576289476, 0.0, 8.327800029206367e-05, 4.132700342114737e-06]
[1977, 0.0, 0]
[0, 0.0, 0.007602492210517841, 0.0003777602252824616, 0.0, 0.007602492210517841, 3.5094704655727327e-06]
[1978, 0.0, 0]
[0.6319632396715502, 0.003419972640218878, 0.008002168880178788, 0.023426339515784128, 0.000286368843069874, 0.0013451175283350776, 5.528531735050119e-05]
[1883, 0.22166666666666665, 0]
[0, 0.0, 0.0010172732593699426, 0.00034183499865682245, 0.0, 2.03059614425325

[1995, 0.044000000000000004, 0]
[0, 0.0, 0.0034704176789521236, 0.00019250669706853227, 0.0, 0.0019694129209163686, 3.330630868857858e-06]
[1996, 0.0, 0]
[0.10072190921749677, 0.006060606060606061, 0.0013936834869183044, 0.00018924599702874854, 0.0001951219512195122, 0.0001045918167222836, 1.5250878894498099e-06]
[1997, 0.0, 0]
[0, 0.0, 0.004850325083379185, 0.0015460650762894178, 0.0, 0.0026422969000298143, 5.425120422781157e-05]
[1998, 0.010666666666666666, 0]
[0, 0.0, 0.004818596257748144, 0.001989704025743615, 0.0, 0.002011022552936266, 3.5496533914873935e-05]
[1999, 0.0, 0]
[0, 0.0, 0.0014240942236416162, 2.361893335720602e-05, 0.0, 0.0007649158898492171, 9.415270478232608e-08]
[2000, 0.0, 0]
[2.0017788693700553, 0.0014169084407259969, 0.0658004933317226, 0.07548063991901652, 3.8336208548974504e-05, 0.0026254339863177496, 8.343479468478268e-06]
[1763, 0.9804999999999999, 1]
[0, 0.0, 0.0033959989848382777, 0.0002640070154448103, 0.0, 0.0011986925287622166, 8.423835848796565e-07]
[1